# Industry Relation

In [19]:
import numpy as np

In [ ]:
# Set the stock market (NASDAQ or NYSE)
market = "NASDAQ"

In [20]:
a = np.load("../Temporal_Relational_Stock_Ranking/data/relation/sector_industry/{}_industry_relation.npy".format(market))

In [21]:
a.shape

(1026, 1026, 97)

## Consistency Tests

### R(N) needs to have at least one value 1

In [22]:
for i in range(a.shape[0]):
    sum = np.sum(a[i][i])
    if sum <= 0:
        print("ERROR: condition not met!")

### R(A)∩R(B) = R(B)∩R(A)

In [23]:
for i in range(a.shape[0]):
    for j in range(a.shape[1]):
        equal = np.array_equal(a[i][j], a[j][i])
        if not equal:
            print("ERROR: Not equal!")

### R(A)∩R(B) ⊂ R(A)

In [24]:
for i in range(a.shape[0]):
    for j in range(a.shape[1]):
        result_1 = a[i][j] * a[i][i]
        result_2 = a[j][i] * a[i][i]
        if not np.array_equal(result_1, a[i][j]):
            print("ERROR: Not equal!")
        if not np.array_equal(result_2, a[j][i]):
            print("ERROR: Not equal!")

## Conversion to edge_index

In [25]:
# function to add edge to edge_index
def add_edge(edge_index, orig_node, dest_node, undirected=True):
    array_to_add = np.array([[orig_node], [dest_node]])
    if edge_index is None:
        new_edge_index = array_to_add.copy()
    else:
        new_edge_index = np.hstack((edge_index, array_to_add))
    if undirected:
        array_to_add = np.array([[dest_node], [orig_node]])
        new_edge_index = np.hstack((new_edge_index, array_to_add))
    return new_edge_index

In [26]:
# loop array
edges = 0
edge_index = None
for i in range(a.shape[0]):
    for j in range(i + 1, a.shape[1]):
        if a[i][j].sum() > 0:
            edge_index = add_edge(edge_index, i, j)
            edges += 1

print(edges)
print(edge_index.shape)
print(np.max(edge_index))

26293
(2, 52586)
1025


## Save

In [27]:
np.save("../relational_data/edge_indexes/{}_sector_industry_edge_index.npy".format(market), edge_index)

## Visualization

In [19]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor(edge_index, dtype=torch.long)
x = torch.zeros(1, a.shape[0])

data = Data(x=x, edge_index=edge_index, num_nodes=a.shape[0])

In [20]:
import networkx as nx
from torch_geometric.utils import to_networkx

g = to_networkx(data, to_undirected=True)
nx.write_gexf(g, "../relational_data/gephi_visualizations/{}_sector_industry.gexf".format(market))